In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!git clone https://github.com/long-practice/mlflow-basecode.git

Cloning into 'mlflow-basecode'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 139 (delta 64), reused 91 (delta 31), pack-reused 0
Receiving objects: 100% (139/139), 135.52 KiB | 7.13 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [4]:
%cd mlflow-basecode

/content/mlflow-basecode


In [5]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [6]:
!pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.0 MB/s eta 0:00:00


In [7]:
from src.pipeline import Pipeline
from preprocessing.preprocess import Preprocess

In [8]:
pipe = Pipeline(train, test)
pipe.run()

INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Remove Outlier
INFO:preprocess:column name is : id
INFO:preprocess:lower_bound is : -2703.0
INFO:preprocess:upper_bound is : 8109.0
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : CementComponent
INFO:preprocess:lower_bound is : -28.25000000000003
INFO:preprocess:upper_bound is : 616.95
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : BlastFurnaceSlag
INFO:preprocess:lower_bound is : -183.89999999999998
INFO:preprocess:upper_bound is : 306.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : FlyAshComponent
INFO:preprocess:lower_bound is : -118.5
INFO:preprocess:upper_bound is : 197.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : WaterComponent
INFO:preprocess:lower_bound is : 149.75
INFO:preprocess

In [10]:
import pickle

xgb_model = None
with open('./artifact/XGBoost.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

lgbm_model = None
with open('./artifact/LightGBM.pkl', 'rb') as f:
    lgbm_model = pickle.load(f)

submission = pd.read_csv('./data/sample_submission.csv')
# submission['Strength'] = xgb_model.predict(test)
submission['Strength'] = lgbm_model.predict(test)

In [11]:
submission

,id,Strength
0,5407,53.359864
1,5408,21.634279
2,5409,36.265433
3,5410,53.983599
4,5411,37.044718
...,...,...
3600,9007,34.992968
3601,9008,37.459238
3602,9009,41.356865
3603,9010,39.419332


In [12]:
import os

n = 0
submission_file_name_list = sorted([fname for fname in os.listdir('./prediction/') if fname.split('.')[-1] == 'csv'])
if submission_file_name_list:
    remove_tail = submission_file_name_list[-1].rstrip('.csv')
    n = int(remove_tail.split('_')[-1])
    n += 1

submission_path = './prediction/'
save_file_name = f'submission_{n}.csv'
file_path = os.path.join(submission_path, save_file_name)
submission.to_csv(file_path, index=False)